# Segmenting and Clustering Neighborhoods in Toronto

#### Importing important importable libraries

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import geocoder
from geopy.geocoders import Nominatim
import folium
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

### Part 1: Parsing the website

#### Creating a local version of the website 

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('local.html', 'w', encoding="utf-8") as fo:
    fo.write(article)

#### Load article, turn into soup and get the tables

In [3]:
article = open('local.html',encoding="utf-8").read()
soup = BeautifulSoup(article, 'lxml')
tables = soup.find_all('table', class_='sortable')

#### Search through the tables for the one with the headings we want

In [4]:
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

#### Defining an empty pandas dataframe for appending

In [5]:
df = pd.DataFrame([],columns=['Postcode', 'Borough', 'Neighbourhood'])

#### Extract the columns we want to a pandas dataframe

In [6]:
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    postcode, borough, neighbourhood = [td.text.strip() for td in tds[:3]]
    df = pd.concat([df,pd.DataFrame([[postcode,borough,neighbourhood]],columns=['Postcode', 'Borough', 'Neighbourhood'])],axis=0)

df.reset_index(inplace=True)
df.drop('index', axis=1,inplace=True)
df = df.rename(columns={'Postcode':'PostalCode', 'Borough':'Borough', 'Neighbourhood':'Neighborhood'})

#### Data cleaning and wrangling

In [7]:
# ignore cells with a borough that is not assigned
df = df[df['Borough'] != 'Not assigned'] 
df.reset_index(inplace=True)
df.drop('index', axis=1,inplace=True)

# fill cells with missing neighborhoods
df['Neighborhood'][df['Neighborhood'] == 'Not assigned'] = df['Borough']  
df.reset_index(inplace=True)
df.drop('index', axis=1,inplace=True)

# unite neighborhoods under same postal codes
i = 0
new_shape = df.shape[0]-1  
while i<new_shape:
    if df.loc[i,'PostalCode'] == df.loc[i+1,'PostalCode']:
        df.loc[i,'Neighborhood'] = df.loc[i,'Neighborhood']+', '+df.loc[i+1,'Neighborhood']
        df.drop(df.index[i+1],inplace=True)
        df.reset_index(inplace=True)
        df.drop('index', axis=1,inplace=True)
        new_shape=new_shape-1
    else:
        i=i+1

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [9]:
df.shape

(103, 3)

### Part 2: Longitudes and latitudes

#### Get longitudes and latitudes (geocoder version)

In [10]:
'''
# latitude and longitude dataframes
lat=pd.DataFrame(np.zeros((df.shape[0],1)),columns=['Latitude'])
long=pd.DataFrame(np.zeros((df.shape[0],1)),columns=['Longitude'])

for i in range(0,df.shape[0]):
    
    postal_code = df.loc[i,'PostalCode']
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    lat.loc[i,'Latitude'] = lat_lng_coords[0]
    long.loc[i,'Longitude'] = lat_lng_coords[1]
    
    print(i)
'''

"\n# latitude and longitude dataframes\nlat=pd.DataFrame(np.zeros((df.shape[0],1)),columns=['Latitude'])\nlong=pd.DataFrame(np.zeros((df.shape[0],1)),columns=['Longitude'])\n\nfor i in range(0,df.shape[0]):\n    \n    postal_code = df.loc[i,'PostalCode']\n    \n    # initialize your variable to None\n    lat_lng_coords = None\n\n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n        lat_lng_coords = g.latlng\n\n    lat.loc[i,'Latitude'] = lat_lng_coords[0]\n    long.loc[i,'Longitude'] = lat_lng_coords[1]\n    \n    print(i)\n"

#### Get longitudes and latitudes (csv version)

In [11]:
df2 = pd.read_csv('Geospatial_Coordinates.csv')

# latitude and longitude dataframes
lat=pd.DataFrame(np.zeros((df.shape[0],1)),columns=['Latitude'])
long=pd.DataFrame(np.zeros((df.shape[0],1)),columns=['Longitude'])

for i in range(0,df.shape[0]):
    lat.loc[i,'Latitude']=df2['Latitude'][df.loc[i,'PostalCode']==df2['Postal Code']].values
    long.loc[i,'Longitude']=df2['Longitude'][df.loc[i,'PostalCode']==df2['Postal Code']].values
    
df = pd.concat([df,lat,long],axis=1)  

In [12]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [13]:
df.shape

(103, 5)

### Part 3: Clustering

#### Finding the coordinates of Toronto

In [14]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'NUKT400LTRSZBTM3J5VME30WD5GSPPHU5JFWIBC2GSML3NOY' # your Foursquare ID
CLIENT_SECRET = 'SD4JXZFMC22RZDQKJM3X5OKIDTPDQH5JX2NJ33P2CGGIRFX3' # your Foursquare Secret
VERSION = '20191021' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NUKT400LTRSZBTM3J5VME30WD5GSPPHU5JFWIBC2GSML3NOY
CLIENT_SECRET:SD4JXZFMC22RZDQKJM3X5OKIDTPDQH5JX2NJ33P2CGGIRFX3


#### Category function

In [17]:
# function that extracts the category of the venue

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Running the category function on each postal code

In [18]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(2275, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


#### Analyze each postal code

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.reindex(columns= cols)

#### Grouping categories and taking their mean

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [22]:
toronto_grouped.shape

(100, 279)

#### Top 10 most common venues

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4       Restaurant  0.03


----Agincourt----
                venue  freq
0      Sandwich Place   0.2
1        Skating Rink   0.2
2      Breakfast Spot   0.2
3              Lounge   0.2
4  Chinese Restaurant   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.18
1              Pharmacy  0.09
2  Fast Food Restaurant  0.09
3           Pizza Place  0.09
4            Beer Store  0.09


----Alderwood,

                  venue  freq
0         Grocery Store   0.2
1          Liquor Store   0.2
2    Athletics & Sports   0.2
3  Gym / Fitness Center   0.2
4        Discount Store   0.2


----Downsview West----
               venue  freq
0      Grocery Store  0.29
1               Bank  0.14
2  Convenience Store  0.14
3               Park  0.14
4              Hotel  0.14


----Downsview, North Park, Upwood Park----
                        venue  freq
0                      Bakery  0.25
1  Construction & Landscaping  0.25
2                        Park  0.25
3            Basketball Court  0.25
4           Accessories Store  0.00


----East Birchmount Park, Ionview, Kennedy Park----
               venue  freq
0     Discount Store  0.50
1   Department Store  0.25
2        Coffee Shop  0.25
3  Mobile Phone Shop  0.00
4      Moving Target  0.00


----East Toronto----
                       venue  freq
0          Convenience Store  0.33
1                Coffee Shop  0.33
2                       Park

                       venue  freq
0       Fast Food Restaurant   0.5
1                 Print Shop   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4                      Motel   0.0


----Runnymede, Swansea----
                venue  freq
0         Coffee Shop  0.09
1                Café  0.09
2         Pizza Place  0.06
3    Sushi Restaurant  0.06
4  Italian Restaurant  0.06


----Ryerson, Garden District----
                       venue  freq
0             Clothing Store  0.08
1                Coffee Shop  0.08
2             Cosmetics Shop  0.04
3  Middle Eastern Restaurant  0.03
4                       Café  0.03


----Scarborough Village----
                       venue  freq
0                 Playground   0.5
1             Cosmetics Shop   0.5
2  Middle Eastern Restaurant   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----Silver Hills, York Mills----
                             venue  freq
0                             Pa

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Sushi Restaurant,Hotel,Asian Restaurant,American Restaurant,Steakhouse,Restaurant
1,Agincourt,Sandwich Place,Chinese Restaurant,Lounge,Breakfast Spot,Skating Rink,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Sandwich Place,Discount Store,Fast Food Restaurant,Liquor Store,Beer Store,Japanese Restaurant,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pub,Skating Rink,Gym,Pharmacy,Sandwich Place,Empanada Restaurant,Electronics Store,Eastern European Restaurant


#### Clustering (k-means)

In [26]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 2, 4, 4, 4, 4, 4, 4, 4])

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.fillna(5,inplace=True) # assign postal codes with no venues to cluster 5


#### Visualization

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
toronto_merged.shape

(103, 16)